In [ ]:
import numpy as np

In [ ]:
def binomial_european_call(S0, K, T, r, sigma, q, N):
    """European call price based on an N-step binomial tree"""
    
    # Calculate time step
    deltaT = T / float(N)

    # Calculate up and down factors
    u = np.exp(sigma*np.sqrt(deltaT))
    d = 1.0 / u

    # Calculate risk-neutral probability
    p = (np.exp((r-q)*deltaT)-d) / (u-d)

    # probabilities * discounting factors
    # Calculate risk-neutral discount factors
    piu = np.exp(-(r-q)*deltaT) * p
    pid = np.exp(-(r-q)*deltaT) * (1-p)

    #Create price matrix 
    C = np.zeros((N+1, N+1))
    S = np.zeros((N+1, N+1))

    # Populate price matrix 
    for i in range(N+1):
        for j in range(i, N+1):
            S[i, j] = S0 * u**(j-i) * d**(i)
    S = np.triu(S)

    # Add payoffs at maturity
    C[:, N] = np.maximum(0, S[:, N]-K)

    # Work backwards to calculate option value at each node
    for j in range(N-1, -1, -1):
        C[:j+1, j] = piu * C[:j+1, j+1] + pid * C[1:j+2, j+1] 

    # Return initial call option value
    return[0,0]

In [ ]:
def binomial_american_call(S0, K, T, r, sigma, q, N):
    deltaT = T / N
    u = np.exp(sigma * np.sqrt(deltaT)) 
    d = 1.0 / u
    p = (np.exp((r-q)*deltaT) - d) / (u - d)

    piu = np.exp(-r * deltaT) * p
    pid = np.exp(-r * deltaT) * (1-p)

    C = np.zeros((N+1, N+1))
    S = np.zeros((N+1, N+1))

    for i in range(N+1):
        for j in range(i+1):
            S[i,j] = S0 * u**(j) * d**(i-j)

    C[:, N] = np.maximum(S[:, N] - K, 0)  

    for j in range(N-1, -1, -1):
        C[:-1, j] = np.maximum(S[:-1, j] - K, 
                        piu * C[:-1, j+1] + pid * C[1:, j+1])
    
        C[-1, j] = np.maximum(S[-1, j] - K, 
                            piu * C[-1, j+1])
                        
    return C[0, 0]